In [ ]:
# Change these values to add and num hidden layers, 
# Total layers=hidden+input+output
num_hidden_layers=10
num_neurons=500
#
#
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import random_split
from matplotlib.pyplot import *
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
import time
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
device

In [ ]:
#https://shashikachamod4u.medium.com/excel-csv-to-pytorch-dataset-def496b6bcc1
# import os
# import pandas as pd
from torch.utils.data import Dataset

class EQ_dataset(Dataset):
    def __init__(self,file_name_inp,file_name_out,file_name_test_in,file_name_test_out):
        X=pd.read_csv(file_name_inp,header=None)
        y=pd.read_csv(file_name_out,header=None)
        
        X_test=pd.read_csv(file_name_test_in,header=None)
        y_test=pd.read_csv(file_name_test_out,header=None)
        
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_test_scaled=scaler.transform(X_test)
        
        self.X_train= torch.tensor(X_scaled[:],dtype=torch.float32)
        self.y_train= torch.tensor(y[:].values,dtype=torch.float32)
        self.X_test= torch.tensor(X_test_scaled[:50],dtype=torch.float32) # use it for nep 3
        self.y_test= torch.tensor(y_test.values[:50],dtype=torch.float32)
#         self.X_test= torch.tensor(X_test_scaled,dtype=torch.float32) 
#         self.y_test= torch.tensor(y_test.values,dtype=torch.float32)
#         print(np.shape(self.X_train))
    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, idx):
        return self.X_train[idx], self.y_train[idx]
    
    
# EQ_set=EQ_dataset('IO_files/Input_real_eq_all_non_lin_large_accel_noPGA.csv','IO_files/Output_real_eq_all_non_lin_large_accel.csv','IO_files/ValidationX6_accel_noPGA.csv','IO_files/ValidationY6_accel.csv')
EQ_set=EQ_dataset('IO_files/Input_real_eq_all_non_lin_large_accel_noPGA.csv','IO_files/Output_real_eq_all_non_lin_large_accel.csv','IO_files/ValidationX_nep3_accel.csv','IO_files/ValidationY_nep3_accel.csv')
splitsize=0.1
train_set, val_set= random_split(EQ_set, [int(np.ceil((1-splitsize)*len(EQ_set))), int(np.floor(splitsize*len(EQ_set)))])
# splitsize=0.1
# train_set, val_set= random_split(train_val_set, [int((1-splitsize)*len(train_val_set)), int(splitsize*len(train_val_set))])
train_loader = torch.utils.data.DataLoader(train_set,batch_size=50, shuffle=True, num_workers=50)
val_loader = torch.utils.data.DataLoader(val_set,batch_size=50, shuffle=True, num_workers=50)



In [ ]:

num_out=len(EQ_set.y_test.T)
num_inputs=len(EQ_set.X_test.T)

class SimpleNet(nn.Module):
    def __init__(self,num_neurons,num_hidden_layers,num_inputs,num_out):
        super(SimpleNet, self).__init__() # takes the nn.module as super
        
        self.num_inputs=num_inputs
        self.num_out=num_out
        
        self.num_neurons=num_neurons
        self.num_hidden_layers=num_hidden_layers
        
        
        self.fc0 = nn.Linear(num_inputs, num_neurons) # first layer
        
        for ct in range(1,num_hidden_layers+1):
            exec("self.fc%d = nn.Linear(num_neurons, num_neurons)"%ct)
        exec("self.fc%d = nn.Linear(num_neurons, num_out)"%(ct+1))

    def forward(self, x):
        x = x
        for ct in np.arange(0,self.num_hidden_layers+1):
            x=eval('F.relu(self.fc%d(x))'%ct)
#             print('F.relu(self.fc%d(x))'%ct)
#             print(ct)
#             print(x.shape)
#             print(x)
        x = eval('self.fc%d(x)'%(ct+1))
#         print('self.fc%d(x)'%(ct+1))
#         print(ct+1)
#         print(x.shape)
#         print(x)
        return x


simplenet = SimpleNet(num_neurons,num_hidden_layers,num_inputs,num_out)
print(simplenet)
simplenet.to(device)




In [ ]:
N_epochs=200
tol=0.2
loss_function = nn.MSELoss()

def loss_function_custom(output, labels):
    y=labels
    y_predict=output
    return loss_function(y[:,1]/torch.mean(y[:,1]),y_predict[:,1]/torch.mean(y[:,1]))+loss_function(y[:,0]/torch.mean(y[:,0]),y_predict[:,0]/torch.mean(y[:,0]))
    




#optimizer = torch.optim.SGD(simplenet.parameters(), lr=1e-4, momentum=0.9)
#optimizer = torch.optim.Adam(simplenet.parameters(), lr=1e-4, )
optimizer = torch.optim.Adam(simplenet.parameters(), lr=1e-4, )

elapsed_time_fl=0

check =0

for epoch in range(N_epochs):
    print('####################################################################################################')
    start = time.time()
    train_loss = 0.0
    val_loss=0.0
    simplenet.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = simplenet(inputs)
        loss = loss_function(outputs, labels)  
#         loss = loss_function_custom(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()


    
    simplenet.eval() 
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = simplenet(inputs)
#         loss = loss_function(outputs, labels)
        loss = loss_function_custom(outputs, labels)
        val_loss += loss.item()
    print("Epoch: {} Train Loss: {} Val Loss: {}".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader)))
    if epoch==0:
        train_loss_prev=train_loss/len(train_loader)
        val_loss_prev=val_loss/len(val_loader)
        
    del_train_loss=train_loss_prev-train_loss/len(train_loader)
    del_val_loss=val_loss_prev-val_loss/len(val_loader)
    
    train_loss_prev=train_loss/len(train_loader)
    val_loss_prev=val_loss/len(val_loader)
    print('check=',check)
    print('Change in train loss=',del_train_loss)
    print('Change in val loss=',del_val_loss)
    elapsed_time_fl = elapsed_time_fl+(time.time() - start) 
    print("Elapsed Time:",elapsed_time_fl)
    print('####################################################################################################')
    if del_val_loss<tol*np.abs(del_train_loss):
        check+=1
    if check>2 and del_val_loss>tol*np.abs(del_train_loss):
        check=0
    if check>3:
        print('Overfitting')
        break
    
    